<a href="https://colab.research.google.com/github/hamidzangiabadi/sailfish-optimization-algorithm/blob/main/Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install ucimlrepo
# !pip install mealpy
# !pip install scikit-learn
!pip install uci-dataset

# import datasets

In [1]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

# fetch dataset
# iris = fetch_ucirepo(id=53) 
# breast_cancer = fetch_ucirepo(id=14)
# communities_and_crime = fetch_ucirepo(id=183) 
# zoo = fetch_ucirepo(id=111)
# automobile = fetch_ucirepo(id=10)
# lymphography = fetch_ucirepo(id=63)
# student_performance = fetch_ucirepo(id=320) 
# ionosphere = fetch_ucirepo(id=52)
# hepatitis = fetch_ucirepo(id=46) 
# letter_recognition = fetch_ucirepo(id=59)
# heart_disease = fetch_ucirepo(id=45) 
# tic_tac_toe_endgame = fetch_ucirepo(id=101) 
# cervical_cancer_behavior_risk = fetch_ucirepo(id=537) 
# connectionist_bench_sonar_mines_vs_rocks = fetch_ucirepo(id=151) 
statlog_heart = fetch_ucirepo(id=145) 
# poker_hand = fetch_ucirepo(id=158) 

print("imported")

Datasets = [
            # { "name" : "Iris" , "dataset" : iris },
            # { "name" : "Breast Cancer" , "dataset" : breast_cancer },
            # { "name" : "Zoo" , "dataset" : zoo },
            # { "name" : "Automobile" , "dataset" : automobile },
#             { "name" : "Lymphography" , "dataset" : lymphography },
            # { "name" : "Student Performance" , "dataset" : student_performance },
            # { "name" : "Ionosphere" , "dataset" : ionosphere },
#             { "name" : "Cervical Cancer Behavior Risk" , "dataset" : cervical_cancer_behavior_risk },
            # { "name" : "Hepatitis" , "dataset" : hepatitis },
            # { "name" : "Communities and Crime" , "dataset" : communities_and_crime },
            # { "name" : "Letter Recognition" , "dataset" : letter_recognition },
            # { "name" : "Heart Disease" , "dataset" : heart_disease },
            # { "name" : "Tic-Tac-Toe Endgame" , "dataset" : tic_tac_toe_endgame },
#             { "name" : "Connectionist Bench" , "dataset" : connectionist_bench_sonar_mines_vs_rocks },
#             { "name" : "HeartEW", "dataset" : statlog_heart}
              { "name" : "Poker Hand", "dataset" : poker_hand}
           ]


dataset = poker_hand


imported


## Preprocessing



In [2]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

def encode_categorical_features(df):
    label_encoder = LabelEncoder()
    object_columns = df.select_dtypes(include=['object']).columns
    
    for column in object_columns:
        df[column + '_encoded'] = label_encoder.fit_transform(df[column])
    
    df.drop(columns=object_columns, inplace=True)

def preprocess_data(dataset):
    encode_categorical_features(dataset)
    encode_categorical_features(dataset)


preprocess_data(dataset.data.features)



data = pd.concat([dataset.data.features, dataset.data.targets], axis=1)
data.dropna(inplace=True)
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]
dataset.data.targets = y
dataset.data.features = X


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12508\135025821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=object_columns, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12508\135025821.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=object_columns, inplace=True)


## Some required functions

In [3]:
import csv

def binary_conversion(X, dim):
    Xbin = np.zeros(dim)
    for d in range(dim):
        if X[d] > 0.5:
            Xbin[d] = 1
        else:
            Xbin[d] = 0

    return Xbin


def save_results(g_best, method_name, num_features,datasetName):
    
    selectedFeaturesSubset = binary_conversion(g_best.solution,num_features)
    featuresCount = int(sum(selectedFeaturesSubset))
    bestFitness = g_best.target.fitness
    
    print(f"Solution: {str(selectedFeaturesSubset)}, Fitness: {g_best.target.fitness}")
    print(f"No of features: {str(featuresCount)}")
    print(f"Reached accuracy: {str(checkAccuracy(selectedFeaturesSubset))}");
    
    acc = checkAccuracy(selectedFeaturesSubset)
    res = {
        "Method": method_name,
        "NumberOfFeatures": str(featuresCount),
        "Accuracy": acc,
        "Error":1-acc,
        "Fitness": bestFitness
    }
    # Save to CSV or any other preferred format
    # Example: Using pandas to save as CSV
    import pandas as pd
    df = pd.DataFrame([res])
    df.to_csv(f"{datasetName}_results.csv", mode='a', index=False, header=not pd.io.common.file_exists(f"{datasetName}_results.csv"))
    

## Evaluate the methods

In [4]:
import numpy as np
from mealpy import FloatVar, GA,SFO,WOA,EHO,GWO,DO,MVO,MFO
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


def checkAccuracy(features):
    max_feat = len(features)
    num_feat = np.sum(binary_conversion(features,num_features) == 1)
    if num_feat == 0 or num_feat == max_feat:
        return 1  

    targets = np.array(y).ravel()
    selected_features = binary_conversion(features,num_features);
    f =  X.iloc[:, selected_features == 1]
    X_train, X_test, y_train, y_test = train_test_split(f, targets, test_size=0.3)
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train.ravel())
    # Predict on test set
    y_pred = svm.predict(X_test)
    Ytest = np.array(y_test)
    num_valid = np.size(X_test, 0)
    yvalid  = Ytest.reshape(num_valid)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

def objective_function(features, X_train, X_test, y_train, y_test):
    max_feat = len(features)
    num_feat = np.sum(binary_conversion(features, num_features) == 1)
    
    # If no features or all features are selected, return a poor score
    if num_feat == 0 or num_feat == max_feat:
        return float('inf')  # Adjusted to return a poor score

    alpha = 0.6
    beta = 0.4

    selected_features = binary_conversion(features, X_train.shape[1])
    f_train = X_train.iloc[:, selected_features == 1]
    f_test = X_test.iloc[:, selected_features == 1]
    
    svm = SVC(kernel='linear')
    svm.fit(f_train, np.array(y_train).ravel())
    y_pred = svm.predict(f_test)

    accuracy = accuracy_score(y_test, y_pred)
    cost = (alpha * (1 - accuracy)) + (beta * (num_feat / max_feat))
    return cost



for dataset in Datasets:
    X = dataset["dataset"].data.features
    y = dataset["dataset"].data.targets

    num_features = X.shape[1]
    search_space = np.array([0, 1] * num_features)

    lb = np.zeros(num_features)
    ub = np.ones(num_features)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
   
    
    problem_dict = {
    "bounds": FloatVar(lb=lb, ub=ub, ),
    "obj_func": lambda features: objective_function(features, X_train, X_test, y_train, y_test),
    "minmax": "min",
    }
    
    for i in range(10):
    
        

    
        SFOModel = SFO.OriginalSFO(epoch=30,pop_size=50, pp = 0.1, AP =4 ,epsilon = 0.01)
        g_best = SFOModel.solve(problem_dict)
        save_results(g_best,"SFO", num_features, dataset["name"])
        
        
        GAmodel = GA.BaseGA(epoch=30,pop_size=50,pc=0.7, pm=0.05)
        g_best = GAmodel.solve(problem_dict)
        save_results(g_best,"GA", num_features, dataset["name"])
        
        EHOModel = EHO.OriginalEHO(epoch=30,pop_size=50, alpha = 0.5, beta = 0.5, n_clans = 2)
        g_best = EHOModel.solve(problem_dict)
        save_results(g_best,"EHO", num_features, dataset["name"])

        GWOModel = GWO.OriginalGWO(epoch=30,pop_size=50)
        g_best = GWOModel.solve(problem_dict)
        save_results(g_best,"GWO", num_features, dataset["name"])

        WOAModel = WOA.OriginalWOA(epoch=30,pop_size=50)
        g_best = WOAModel.solve(problem_dict)
        save_results(g_best,"WOA", num_features, dataset["name"])

        MFOModel = MFO.OriginalMFO(epoch=30,pop_size=50)
        g_best = MFOModel.solve(problem_dict)
        save_results(g_best,"MFO", num_features, dataset["name"])


2024/05/22 02:04:18 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: Solving single objective optimization problem.
2024/05/22 02:05:12 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 1, Current best: 0.17150997150997152, Global best: 0.17150997150997152, Runtime: 5.33749 seconds
2024/05/22 02:05:17 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 2, Current best: 0.17150997150997152, Global best: 0.17150997150997152, Runtime: 4.80557 seconds
2024/05/22 02:05:19 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 3, Current best: 0.17150997150997152, Global best: 0.17150997150997152, Runtime: 2.57100 seconds
2024/05/22 02:05:25 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 4, Current best: 0.17150997150997152, Global best: 0.17150997150997152, Runtime: 5.54628 seconds
2024/05/22 02:05:27 AM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 5, Current best: 0.17150997150997152, Global best: 0.171509

Solution: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.], Fitness: 0.17150997150997152
No of features: 1
Reached accuracy: 0.691358024691358


2024/05/22 02:06:51 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 1, Current best: 0.22678062678062677, Global best: 0.2245014245014245, Runtime: 3.53722 seconds
2024/05/22 02:06:53 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 2, Current best: 0.19487179487179487, Global best: 0.19487179487179487, Runtime: 2.28700 seconds
2024/05/22 02:06:54 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 3, Current best: 0.17150997150997152, Global best: 0.17150997150997152, Runtime: 0.91477 seconds
2024/05/22 02:06:54 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 4, Current best: 0.17150997150997152, Global best: 0.17150997150997152, Runtime: 0.40553 seconds
2024/05/22 02:06:55 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 5, Current best: 0.17150997150997152, Global best: 0.17150997150997152, Runtime: 0.30017 seconds
2024/05/22 02:06:55 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>P

Solution: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.], Fitness: 0.17150997150997152
No of features: 1
Reached accuracy: 0.7407407407407407


2024/05/22 02:07:12 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 1, Current best: 0.23304843304843306, Global best: 0.23304843304843306, Runtime: 4.90542 seconds
2024/05/22 02:07:16 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 2, Current best: 0.21082621082621084, Global best: 0.21082621082621084, Runtime: 4.64880 seconds
2024/05/22 02:07:19 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 3, Current best: 0.2022792022792023, Global best: 0.2022792022792023, Runtime: 2.69048 seconds
2024/05/22 02:07:21 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 4, Current best: 0.17150997150997152, Global best: 0.17150997150997152, Runtime: 2.44626 seconds
2024/05/22 02:07:25 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 5, Current best: 0.17150997150997152, Global best: 0.17150997150997152, Runtime: 3.46994 seconds
2024/05/22 02:07:28 AM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem:

...

Solution: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.], Fitness: 0.17150997150997152
No of features: 1
Reached accuracy: 0.7160493827160493


2024/05/22 02:53:30 AM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 1, Current best: 0.20854700854700853, Global best: 0.20854700854700853, Runtime: 3.49107 seconds
2024/05/22 02:53:31 AM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 2, Current best: 0.20854700854700853, Global best: 0.20854700854700853, Runtime: 1.26041 seconds
2024/05/22 02:53:34 AM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 3, Current best: 0.20854700854700853, Global best: 0.20854700854700853, Runtime: 2.65531 seconds
2024/05/22 02:53:35 AM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 4, Current best: 0.20854700854700853, Global best: 0.20854700854700853, Runtime: 1.33054 seconds
2024/05/22 02:53:36 AM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Problem: P, Epoch: 5, Current best: 0.20854700854700853, Global best: 0.20854700854700853, Runtime: 1.14958 seconds
2024/05/22 02:53:39 AM, INFO, mealpy.swarm_based.WOA.OriginalWOA: >>>Proble

Solution: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.], Fitness: 0.20854700854700853
No of features: 1
Reached accuracy: 0.7530864197530864


2024/05/22 02:54:08 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem: P, Epoch: 1, Current best: 0.22678062678062677, Global best: 0.22678062678062677, Runtime: 5.50313 seconds
2024/05/22 02:54:12 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem: P, Epoch: 2, Current best: 0.19487179487179487, Global best: 0.19487179487179487, Runtime: 4.31560 seconds
2024/05/22 02:54:15 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem: P, Epoch: 3, Current best: 0.19487179487179487, Global best: 0.19487179487179487, Runtime: 3.16542 seconds
2024/05/22 02:54:18 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem: P, Epoch: 4, Current best: 0.19487179487179487, Global best: 0.19487179487179487, Runtime: 3.16673 seconds
2024/05/22 02:54:21 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Problem: P, Epoch: 5, Current best: 0.19487179487179487, Global best: 0.19487179487179487, Runtime: 3.15030 seconds
2024/05/22 02:54:23 AM, INFO, mealpy.swarm_based.MFO.OriginalMFO: >>>Proble

Solution: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.], Fitness: 0.17150997150997152
No of features: 1
Reached accuracy: 0.7407407407407407


# create plots